In [ ]:
import torch
from unsloth import FastLanguageModel
from transformers import pipeline
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [13]:
import torch
print(torch.cuda.is_available())  # Should return True


False


In [ ]:

# Define the list of 4-bit quantized models
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
    "unsloth/Llama-3.2-1B-bnb-4bit",
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"
]

# Load dataset
ds = load_dataset("NebulaByte/E-Commerce_Customer_Support_Conversations")
df = pd.DataFrame(ds['train'])

# Split the dataset into training and testing sets
df = df[["customer_sentiment", "conversation"]]
X_train = []
X_test = []
for sentiment in ['neutral', 'negative', 'frustrated', 'positive']:
    train, test = train_test_split(df[df.customer_sentiment == sentiment],
                                   train_size=0.6,
                                   test_size=0.4,
                                   shuffle=True,
                                   random_state=42)
    X_train.append(train)
    X_test.append(test)

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

# Function to generate prompts
def generate_prompt(data_point):
    return f"""
    Based on the following conversation:

    Conversation: "{data_point['conversation']}"

    Determine the overall sentiment of the conversation. Choose from the following options:
    - Neutral
    - Negative
    - Frustrated
    - Positive

    Solution: The correct sentiment is "{data_point['customer_sentiment']}"
    """.strip()

def generate_test_prompt(data_point):
    return f"""
    Based on the following conversation:

    Conversation: "{data_point['conversation']}"

    Determine the overall sentiment of the conversation. Choose from the following options:
    - Neutral
    - Negative
    - Frustrated
    - Positive

    Solution: The correct sentiment is
    """.strip()

# Apply prompt generation
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), columns=["text"])
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

# Function to predict sentiments
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=3,
                        temperature=0.2)
        result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
        answer = result[0]['generated_text'].split("The correct sentiment is")[-1].strip().lower()
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        elif "frustrated" in answer:
            y_pred.append("frustrated")
        else:
            y_pred.append("none")
    return y_pred

# Evaluate each model
for model_name in fourbit_models:
    print(f"Evaluating model: {model_name}")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True
    )
    y_pred = predict(X_test, model, tokenizer)
    # Compare y_pred with actual sentiments in X_test['customer_sentiment']
    # Calculate accuracy or other metrics as needed
